In [11]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import keras
import keras.backend as K
from keras.layers import Flatten, Concatenate, Input , Embedding, LSTM, Dense, Dropout
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras import Sequential
from keras.models import Model

Load IMDB dataset from keras.datasets.

"This is a dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative)." [[1]](https://keras.io/api/datasets/imdb/)

In [12]:
vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size, skip_top=127, oov_char= 00 )

print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))
print('---review---')
print(X_train[6])
print('---label---')
print(y_train[6])

print('Maximum review length: {}'.format(
len(max((X_train + X_test), key=len))))

print('Minimum review length: {}'.format(
len(min((X_train + X_test), key=len))))

Loaded dataset with 25000 training samples, 25000 test samples
---review---
[0, 0, 365, 1234, 0, 1156, 354, 0, 0, 0, 0, 0, 1016, 0, 0, 356, 0, 0, 1349, 500, 746, 0, 200, 0, 4132, 0, 0, 0, 1117, 1831, 0, 0, 4831, 0, 0, 0, 4183, 0, 369, 0, 215, 1345, 143, 0, 0, 1838, 0, 1974, 0, 0, 0, 257, 0, 0, 486, 0, 0, 0, 0, 0, 271, 0, 196, 0, 949, 4121, 0, 0, 0, 0, 2212, 2436, 819, 0, 0, 0, 0, 180, 0, 227, 0, 0, 2494, 0, 0, 423, 0, 168, 0, 0, 0, 0, 0, 665, 0, 270, 0, 0, 0, 197, 0, 161, 0, 0, 0, 0, 0, 0, 419, 665, 0, 0, 0, 0, 0, 0, 2084, 0, 4773, 0, 0, 0, 1901]
---label---
1
Maximum review length: 2697
Minimum review length: 70


Change sequences length in order to be in a same size.

In [13]:
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

print('shape train:',X_train.shape, 'shape test:', X_test.shape)

shape train: (25000, 500) shape test: (25000, 500)


Define layers inside the model.

In [14]:
input_seq = Input(shape=(max_words,))
embedding_size= 32
emb = Embedding(vocabulary_size, embedding_size)(input_seq)

In [15]:
input_seq

<tf.Tensor 'input_2:0' shape=(None, 500) dtype=float32>

In [16]:
emb

<tf.Tensor 'embedding_2/embedding_lookup/Identity_1:0' shape=(None, 500, 32) dtype=float32>

In [17]:
# ---------------- path one -------------------------
path_1 = LSTM(40, return_sequences= True)(emb)
print('p1:')
print(path_1)

path_1= LSTM(20, return_sequences= True)(path_1)
print('p1:')
print(path_1)

# ---------------- path two -------------------------
path_2 = LSTM(40, return_sequences= True)(emb)
print('p2:')
print(path_2)
path_2 = LSTM(20, return_sequences= True)(path_2)
print('p2:')
print(path_2)

# ---------------- concatenating -------------------------
outl = Concatenate()([path_1,path_2])
print('out:')
print(outl)

outl = Flatten()(outl)
print('out after flatten:')
print(outl)

# ---------------- final output -------------------------
outl = Dense(1, activation='sigmoid')(outl)
print('final out')
outl

p1:
Tensor("lstm_5/transpose_1:0", shape=(None, 500, 40), dtype=float32)
p1:
Tensor("lstm_6/transpose_1:0", shape=(None, 500, 20), dtype=float32)
p2:
Tensor("lstm_7/transpose_1:0", shape=(None, 500, 40), dtype=float32)
p2:
Tensor("lstm_8/transpose_1:0", shape=(None, 500, 20), dtype=float32)
out:
Tensor("concatenate_2/concat:0", shape=(None, 500, 40), dtype=float32)
out after flatten:
Tensor("flatten_2/Reshape:0", shape=(None, None), dtype=float32)
final out


<tf.Tensor 'dense_2/Sigmoid:0' shape=(None, 1) dtype=float32>

In [21]:
model= Model(inputs = input_seq, outputs = outl)
print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 500, 32)      160000      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LSTM)                   (None, 500, 40)      11680       embedding_2[0][0]                
__________________________________________________________________________________________________
lstm_7 (LSTM)                   (None, 500, 40)      11680       embedding_2[0][0]                
____________________________________________________________________________________________

Train

In [22]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])
			 
batch_size = 512
num_epochs = 20
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24488 samples, validate on 512 samples
Epoch 1/20
24488/24488 [==============================] - 200s 8ms/step - loss: 0.1548 - accuracy: 0.9407 - precision_3: 0.9378 - recall_3: 0.9440 - val_loss: 0.5494 - val_accuracy: 0.8477 - val_precision_3: 0.8904 - val_recall_3: 0.7930
Epoch 2/20
24488/24488 [==============================] - 197s 8ms/step - loss: 0.0614 - accuracy: 0.9789 - precision_3: 0.9794 - recall_3: 0.9784 - val_loss: 0.7768 - val_accuracy: 0.8398 - val_precision_3: 0.8884 - val_recall_3: 0.7773
Epoch 3/20
24488/24488 [==============================] - 194s 8ms/step - loss: 0.0411 - accuracy: 0.9879 - precision_3: 0.9882 - recall_3: 0.9876 - val_loss: 0.8737 - val_accuracy: 0.8457 - val_precision_3: 0.8766 - val_recall_3: 0.8047
Epoch 4/20
24488/24488 [==============================] - 198s 8ms/step - loss: 0.0282 - accuracy: 0.9919 - precision_3: 0.9918 - recall_3: 0.9919 - val_loss: 1.0651 - val_accuracy: 0.8262 - val_precision_3: 0.8678 - val_recall_3: 0.7695


In [23]:
print("accuracy:", scores[1]*100 , "\n precision:", scores[2]*100, "\n recall:", scores[3]*100)

accuracy: 81.70400261878967 
 precision: 82.20380544662476 
 recall: 80.92799782752991
